In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [ ]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Hybrid import LinearHybridRecommender, LinearZScoreNormalizedHybridRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

model = LinearHybridRecommender
slim = SLIMElasticNetRecommender(URM_train)
rp3 = RP3betaRecommender(URM_train)
itemknn = ItemKNNCFRecommender(URM_train)
slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668)
rp3.fit(topK=32, alpha=0.22028996694465927, beta=0.17339063177346345)
#rp3.fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

SLIMElasticNetRecommender: URM Detected 463 ( 3.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 350 ( 1.6%) items with no interactions.
RP3betaRecommender: URM Detected 463 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 350 ( 1.6%) items with no interactions.
ItemKNNCFRecommender: URM Detected 463 ( 3.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 350 ( 1.6%) items with no interactions.
SLIMElasticNetRecommender: Processed 18239 (82.1%) in 5.00 min. Items per second: 60.78
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 6.50 min. Items per second: 57.01
RP3betaRecommender: Similarity column 22222 (100.0%), 5387.29 column/sec. Elapsed time 4.12 sec
Unable to load Cython Compute_Similarity, reverting to Python


In [3]:
import optuna as op
def objective(trial):
    alphas = [trial.suggest_float("alphas[0]", 0, 2), trial.suggest_float("alphas[1]", 0, 2), trial.suggest_float("alphas[2]", 0, 2)]
    
    recommender = model(URM_train, [slim, rp3, itemknn])
    #recommender.fit(**trial.params)
    recommender.fit(alphas)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [9]:
best_params = {
    "alphas": [1, 1, 1],
}

study_name = "hybrid-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=20)

[I 2023-11-27 14:18:24,976] Using an existing study with name 'hybrid-study' instead of creating a new one.


SLIMElasticNetRP3betaItemKNNCFHybridRecommender: URM Detected 466 ( 3.7%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFHybridRecommender: URM Detected 322 ( 1.4%) items with no interactions.
EvaluatorHoldout: Processed 10039 (100.0%) in 8.89 sec. Users per second: 1129


[I 2023-11-27 14:18:34,067] Trial 181 finished with value: 0.03123723035716291 and parameters: {'alphas[0]': 0.5155126299525674, 'alphas[1]': 0.7773041667745069, 'alphas[2]': 0.7813365086487185}. Best is trial 56 with value: 0.031380586177722464.


SLIMElasticNetRP3betaItemKNNCFHybridRecommender: URM Detected 466 ( 3.7%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFHybridRecommender: URM Detected 322 ( 1.4%) items with no interactions.
EvaluatorHoldout: Processed 10039 (100.0%) in 8.59 sec. Users per second: 1169


[I 2023-11-27 14:18:42,732] Trial 182 finished with value: 0.031181534871145625 and parameters: {'alphas[0]': 0.6737073342130012, 'alphas[1]': 0.45615916812277246, 'alphas[2]': 1.9263708211622386}. Best is trial 56 with value: 0.031380586177722464.


...

[I 2023-11-27 14:21:06,498] Trial 198 finished with value: 0.03124764213219263 and parameters: {'alphas[0]': 0.8043700691616421, 'alphas[1]': 0.8004266498564697, 'alphas[2]': 1.8995165699217158}. Best is trial 56 with value: 0.031380586177722464.


SLIMElasticNetRP3betaItemKNNCFHybridRecommender: URM Detected 466 ( 3.7%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFHybridRecommender: URM Detected 322 ( 1.4%) items with no interactions.
EvaluatorHoldout: Processed 10039 (100.0%) in 8.91 sec. Users per second: 1127


[I 2023-11-27 14:21:15,492] Trial 199 finished with value: 0.031236732299587462 and parameters: {'alphas[0]': 0.5416750616569345, 'alphas[1]': 0.7391222711340067, 'alphas[2]': 1.4328731673124357}. Best is trial 56 with value: 0.031380586177722464.


SLIMElasticNetRP3betaItemKNNCFHybridRecommender: URM Detected 466 ( 3.7%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFHybridRecommender: URM Detected 322 ( 1.4%) items with no interactions.
EvaluatorHoldout: Processed 10039 (100.0%) in 8.91 sec. Users per second: 1127


[I 2023-11-27 14:21:24,478] Trial 200 finished with value: 0.031244321748356337 and parameters: {'alphas[0]': 0.47376152867947885, 'alphas[1]': 0.8554486426773527, 'alphas[2]': 1.7976434192926811}. Best is trial 56 with value: 0.031380586177722464.


In [11]:
study.best_params

{'alphas[0]': 0.706936448151171,
 'alphas[1]': 0.6559700273535034,
 'alphas[2]': 1.8265466200396685}

In [15]:
final = model(URM_train_validation, [slim, rp3, itemknn])
#final.fit(alphas=[3,1.25])
final.fit(alphas=[study.best_params["alphas[0]"],study.best_params["alphas[1]"], study.best_params["alphas[2]"]])
#final.fit(**study.best_params)

SLIMElasticNetRP3betaItemKNNCFHybridRecommender: URM Detected 216 ( 1.7%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFHybridRecommender: URM Detected 94 ( 0.4%) items with no interactions.


In [16]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2184 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10454 (100.0%) in 8.71 sec. Users per second: 1200


(       PRECISION PRECISION_RECALL_MIN_DEN   RECALL      MAP MAP_MIN_DEN  \
 cutoff                                                                    
 10      0.089028                 0.150759  0.12639  0.04558    0.076747   
 
             MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                       ...   
 10      0.26035  0.140004  0.104469  0.493495      0.340601  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.827188          0.408213    0.827188       0.057399   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.156523                   0.997471             0.164554   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                            

In [17]:
#final.fit(alphas=[3,1.25])
evaluator_test.evaluateRecommender(slim)

EvaluatorHoldout: Processed 10454 (100.0%) in 6.85 sec. Users per second: 1526


(       PRECISION PRECISION_RECALL_MIN_DEN   RECALL      MAP MAP_MIN_DEN  \
 cutoff                                                                    
 10      0.079759                 0.137157  0.11639  0.03848    0.066675   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.234882  0.125152  0.094654  0.475703      0.298827  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.827188          0.393496    0.827188       0.050161   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.020847                   0.997362             0.145638   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                         